<a href="https://colab.research.google.com/github/yu0ki/BERT_Practice/blob/main/Chapter9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''
９章でやること：文章校正
BERTによって（固有表現抽出 chap8のように）トークン単位の分類を行い

'''

'\n９章でやること：文章校正\nBERTによって（固有表現抽出 chap8のように）トークン単位の分類を行い\n\n'

In [2]:
# まずはディレクトリ移動
!mkdir chap9
%cd ./chap9


/content/chap9


In [3]:
# 次はライブラリ
# バージョンが教科書のだと古いようなので、模範解答（https://github.com/stockmarkteam/bert-book/blob/master/Chapter6.ipynb）にバージョンを合わせている
!pip install transformers==4.18.0 fugashi==1.1.0 ipadic==1.0.0 pytorch-lightning==1.6.1

# 集合に関する数学的な操作のライブラリかな？
# https://qiita.com/anmint/items/37ca0ded5e1d360b51f3
import itertools

import random
from tqdm import tqdm

# unicode文字列に対していろいろ操作できそうなライブラリ
# https://docs.python.org/ja/3/library/unicodedata.html
import unicodedata

# pandas https://aiacademy.jp/media/?p=152
import pandas as pd
import torch
from torch.utils.data import DataLoader
from transformers import BertJapaneseTokenizer, BertForTokenClassification, BertForMaskedLM
import pytorch_lightning as pl

# 日本語学習済みモデル
MODEL_NAME = 'cl-tohoku/bert-base-japanese-whole-word-masking'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.0 MB 34.0 MB/s 
     |████████████████████████████████| 486 kB 65.5 MB/s 
     |████████████████████████████████| 13.4 MB 23.2 MB/s 
     |████████████████████████████████| 582 kB 57.4 MB/s 
     |████████████████████████████████| 120 kB 64.0 MB/s 
     |████████████████████████████████| 6.6 MB 50.1 MB/s 
     |████████████████████████████████| 880 kB 33.0 MB/s 
     |████████████████████████████████| 419 kB 48.2 MB/s 
  Created wheel for ipadic: filename=ipadic-1.0.0-py3-none-any.whl size=13556723 sha256=93e9201e846c21e3f789d034421b7485605425bf2ab14b9fa1fe83bdab0892db
  Stored in directory: /root/.cache/pip/wheels/33/8b/99/cf0d27191876637cd3639a560f93aa982d7855ce826c94348b
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=5e8c34330ea1c16e7b517e388484043c3ca2f00522a55e19e7fbad7e71cae608
  Stored in dire

In [4]:
'''
今回やるのは、漢字の誤変換を校正するタスク

入力：誤変換を含む文章
出力：入力文章をトークンに分割し、誤変換していないトークンはそのまま出力し、誤変換があったトークンは訂正して出力する

実装方針（BERT部分）：入力されたそれぞれのトークンに対して、BERTの語彙（今回は32000語）の中から、あるべき単語を選ぶ分類問題として実装


注意：
今回は簡単のため、以下のような条件を設ける

例えば、当初という漢字を打とうとして、誤って投書としてしまった場合
トークナイザは「当初」は「当初」、「投書」は「投 + '##書'」というふうに分割してしまう
よって、「投書」という誤りを「当初」に正すには、複雑な処理を要する
今回はこのようなケースは想定せず、単純に正解文と誤り文がきちんと対応してトークン化される場合だけを扱う
'''

"\n今回やるのは、漢字の誤変換を校正するタスク\n\n入力：誤変換を含む文章\n出力：入力文章をトークンに分割し、誤変換していないトークンはそのまま出力し、誤変換があったトークンは訂正して出力する\n\n実装方針（BERT部分）：入力されたそれぞれのトークンに対して、BERTの語彙（今回は32000語）の中から、あるべき単語を選ぶ分類問題として実装\n\n\n注意：\n今回は簡単のため、以下のような条件を設ける\n\n例えば、当初という漢字を打とうとして、誤って投書としてしまった場合\nトークナイザは「当初」は「当初」、「投書」は「投 + '##書'」というふうに分割してしまう\nよって、「投書」という誤りを「当初」に正すには、複雑な処理を要する\n今回はこのようなケースは想定せず、単純に正解文と誤り文がきちんと対応してトークン化される場合だけを扱う\n"

In [5]:
'''
さて、トークナイザの定義に移ろう

分類問題において、トークナイザには以下の３つの関数を実装する（８章でも同じようなことをやった。分類問題のお決まりなのかな）

1. 
誤変換した文章と、正しい文章が与えられた時に、それぞれの文章を符号化して、モデルに入力できる形式にする関数
学習時に使用する

2. 
誤変換した文章を符号化するとともに、各トークンの文章中の位置を特定する関数
推論時、BERTに入力する前に実行

3.
BERTが出力したトークン列が与えられた時、空白を考慮して、それを文章として出力する関数
推論時、BERTからの出力を加工するのに必要

'''

'\nさて、トークナイザの定義に移ろう\n\n分類問題において、トークナイザには以下の３つの関数を実装する（８章でも同じようなことをやった。分類問題のお決まりなのかな）\n\n1. \n誤変換した文章と、正しい文章が与えられた時に、それぞれの文章を符号化して、モデルに入力できる形式にする関数\n学習時に使用する\n\n2. \n誤変換した文章を符号化するとともに、各トークンの文章中の位置を特定する関数\n推論時、BERTに入力する前に実行\n\n3.\nBERTが出力したトークン列が与えられた時、空白を考慮して、それを文章として出力する関数\n推論時、BERTからの出力を加工するのに必要\n\n'

In [6]:
# マジで８章と一緒だと思うので、解答をコピペしてきました・・・
class SC_tokenizer(BertJapaneseTokenizer):
       
    def encode_plus_tagged(
        self, wrong_text, correct_text, max_length=128
    ):
        """
        ファインチューニング時に使用。
        誤変換を含む文章と正しい文章を入力とし、
        符号化を行いBERTに入力できる形式にする。
        """
        # 誤変換した文章をトークン化し、符号化
        encoding = self(
            wrong_text, 
            max_length=max_length, 
            padding='max_length', 
            truncation=True
        )
        # 正しい文章をトークン化し、符号化
        encoding_correct = self(
            correct_text,
            max_length=max_length,
            padding='max_length',
            truncation=True
        ) 
        # 正しい文章の符号をラベルとする
        encoding['labels'] = encoding_correct['input_ids'] 

        return encoding

    def encode_plus_untagged(
        self, text, max_length=None, return_tensors=None
    ):
        """
        文章を符号化し、それぞれのトークンの文章中の位置も特定しておく。
        """
        # 文章のトークン化を行い、
        # それぞれのトークンと文章中の文字列を対応づける。
        tokens = [] # トークンを追加していく。
        tokens_original = [] # トークンに対応する文章中の文字列を追加していく。
        words = self.word_tokenizer.tokenize(text) # MeCabで単語に分割
        for word in words:
            # 単語をサブワードに分割
            tokens_word = self.subword_tokenizer.tokenize(word) 
            tokens.extend(tokens_word)
            if tokens_word[0] == '[UNK]': # 未知語への対応
                tokens_original.append(word)
            else:
                tokens_original.extend([
                    token.replace('##','') for token in tokens_word
                ])

        # 各トークンの文章中での位置を調べる。（空白の位置を考慮する）
        position = 0
        spans = [] # トークンの位置を追加していく。
        for token in tokens_original:
            l = len(token)
            while 1:
                if token != text[position:position+l]:
                    position += 1
                else:
                    spans.append([position, position+l])
                    position += l
                    break

        # 符号化を行いBERTに入力できる形式にする。
        input_ids = self.convert_tokens_to_ids(tokens) 
        encoding = self.prepare_for_model(
            input_ids, 
            max_length=max_length, 
            padding='max_length' if max_length else False, 
            truncation=True if max_length else False
        )
        sequence_length = len(encoding['input_ids'])
        # 特殊トークン[CLS]に対するダミーのspanを追加。
        spans = [[-1, -1]] + spans[:sequence_length-2] 
        # 特殊トークン[SEP]、[PAD]に対するダミーのspanを追加。
        spans = spans + [[-1, -1]] * ( sequence_length - len(spans) ) 

        # 必要に応じてtorch.Tensorにする。
        if return_tensors == 'pt':
            encoding = { k: torch.tensor([v]) for k, v in encoding.items() }

        return encoding, spans

    def convert_bert_output_to_text(self, text, labels, spans):
        """
        推論時に使用。
        文章と、各トークンのラベルの予測値、文章中での位置を入力とする。
        そこから、BERTによって予測された文章に変換。
        """
        assert len(spans) == len(labels)

        # labels, spansから特殊トークンに対応する部分を取り除く
        labels = [label for label, span in zip(labels, spans) if span[0]!=-1]
        spans = [span for span in spans if span[0]!=-1]

        # BERTが予測した文章を作成
        predicted_text = ''
        position = 0
        for label, span in zip(labels, spans):
            start, end = span
            if position != start: # 空白の処理
                predicted_text += text[position:start]
            predicted_token = self.convert_ids_to_tokens(label)
            predicted_token = predicted_token.replace('##', '')
            predicted_token = unicodedata.normalize(
                'NFKC', predicted_token
            ) 
            predicted_text += predicted_token
            position = end
        
        return predicted_text

In [7]:
# 上で作ったトークナイザの動作確認をするぜ
tokenizer = SC_tokenizer.from_pretrained(MODEL_NAME)

Downloading:   0%|          | 0.00/252k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/110 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/479 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertJapaneseTokenizer'. 
The class this function is called from is 'SC_tokenizer'.


In [8]:
# encode_plus_taggedの挙動確認
wrong_text = '優勝トロフィーを変換した'
correct_text = '優勝トロフィーを返還した'

print(tokenizer.encode_plus_tagged(wrong_text, correct_text, max_length=12))

{'input_ids': [2, 759, 18204, 11, 4618, 15, 10, 3, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0], 'labels': [2, 759, 18204, 11, 8274, 15, 10, 3, 0, 0, 0, 0]}


In [9]:
# encode_plus_untaggedの挙動確認
encoding, spans = tokenizer.encode_plus_untagged(wrong_text, return_tensors='pt')

print('# encoding')
print(encoding)
print('# spans')
print(spans)

# encoding
{'input_ids': tensor([[    2,   759, 18204,    11,  4618,    15,    10,     3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}
# spans
[[-1, -1], [0, 2], [2, 7], [7, 8], [8, 10], [10, 11], [11, 12], [-1, -1]]


In [10]:
# convert_bert_output_to_text
predicted_labels = [2, 759, 18204, 11, 8274, 15, 10, 3]
predicted_text = tokenizer.convert_bert_output_to_text(
    wrong_text, predicted_labels, spans
)
print(predicted_text)

優勝トロフィーを返還した


In [11]:
'''
トークナイザが完成したので
BERT部分を実装しよう

８章と同じようにBertForTokenClassificationを使用することも可能だが、
ここではあえて、最初に使用したBertForMaskedLM(穴埋め用のモデル)を使う

理由：
文章校正のタスク（周りの文脈を見て正しい語句を予測する）は
穴埋めのタスク（周りの文章を見て正しい語句を予測する）と酷似している
ゆえに、こちらのモデルの方が、初めからある程度妥当なパラメータが与えられている可能性が高く、
ファインチューニングにかかる時間を短くできる
'''

'\nトークナイザが完成したので\nBERT部分を実装しよう\n\n８章と同じようにBertForTokenClassificationを使用することも可能だが、\nここではあえて、最初に使用したBertForMaskedLM(穴埋め用のモデル)を使う\n\n理由：\n文章校正のタスク（周りの文脈を見て正しい語句を予測する）は\n穴埋めのタスク（周りの文章を見て正しい語句を予測する）と酷似している\nゆえに、こちらのモデルの方が、初めからある程度妥当なパラメータが与えられている可能性が高く、\nファインチューニングにかかる時間を短くできる\n'

In [12]:
# BERTモデルをロード
bert_mlm = BertForMaskedLM.from_pretrained(MODEL_NAME)
bert_mlm  = bert_mlm.cuda()

Downloading:   0%|          | 0.00/424M [00:00<?, ?B/s]

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [13]:
'''
bert_mlmにも学習と推論の２つの使い方がある
推論：符号化した文章を入力し、トークンごとのラベルの分類スコアを出力する
学習：符号化した文章とトークンごとのラベルを入力として、損失の値を出力する

'''

'\nbert_mlmにも学習と推論の２つの使い方がある\n推論：符号化した文章を入力し、トークンごとのラベルの分類スコアを出力する\n学習：符号化した文章とトークンごとのラベルを入力として、損失の値を出力する\n\n'

In [14]:
# 推論
text = '優勝トロフィーを変換した。'

# まずは符号化して位置を特定
encoding, spans = tokenizer.encode_plus_untagged(text, return_tensors='pt')
encoding = { k : v.cuda() for k, v in encoding.items() }

# bertに入力
with torch.no_grad():
    output = bert_mlm(**encoding)
    scores = output.logits
    labels_predicted = scores[0].argmax(-1).cpu().numpy().tolist()


# テキストに戻して出力
text_predicted = tokenizer.convert_bert_output_to_text(text, labels_predicted, spans)
print(text_predicted)

優勝トロフィーを獲得した。


In [16]:
# 学習

# データ定義
data = [
    {
        'wrong_text': '優勝トロフィーを変換した。',
        'correct_text': '優勝トロフィーを返還した。',
    },
    {
        'wrong_text': '人と森は強制している。',
        'correct_text': '人と森は共生している。',
    }
]


# データローダ定義前に、dataを整形
max_length=32
dataset_for_loader = []

for sample in data:
    wrong_text = sample['wrong_text']
    correct_text = sample['correct_text']

    encoding = tokenizer.encode_plus_tagged(
        wrong_text, correct_text, max_length
    )
    encoding = { k : torch.tensor(v) for k, v in encoding.items() }
    dataset_for_loader.append(encoding)


# データローダ設定
dataloader = DataLoader(dataset_for_loader, batch_size=2)

# ミニバッチをBERTへ入力
for batch in dataloader:
    encoding = { k : v.cuda() for k, v in batch.items() }
    output = bert_mlm(**encoding)
    loss = output.loss


In [17]:
# さて、実際データセットを使って学習してみようじゃん
# データセットの詳細：https://nlp.ist.i.kyoto-u.ac.jp/?日本語Wikipedia入力誤りデータセット
!curl -L "https://nlp.ist.i.kyoto-u.ac.jp/DLcounter/lime.cgi?down=https://nlp.ist.i.kyoto-u.ac.jp/nl-resource/JWTD/jwtd.tar.gz&name=JWTD.tar.gz" -o JWTD.tar.gz
!tar zxvf JWTD.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   244  100   244    0     0    383      0 --:--:-- --:--:-- --:--:--   383
100 64.9M  100 64.9M    0     0  12.5M      0  0:00:05  0:00:05 --:--:-- 16.0M
jwtd/
jwtd/train.jsonl
jwtd/test.jsonl


In [18]:
'''
上記データセットにはいろいろな情報が入っているが、今回は漢字誤変換のものだけ取り出して使う。

よって、学習の前の前処理の手順は以下になる

1. categoryがkanji-conversionの行(row)だけを抽出する
2. pre_textをwrong_textへ、post_textをcorrect_textへ名前変更
3. 抽出した文章の正規化を行う(normalize)
4. トークン化した際に、正解の文章と誤りの文章の対応がつく かつ 誤りトークン数が2個以内ものだけを抽出する(check_token_count)

'''

def create_dataset(data_df):

    tokenizer = SC_tokenizer.from_pretrained(MODEL_NAME)

    def check_token_count(row):
        """
        誤変換の文章と正しい文章でトークンに対応がつくかどうかを判定。
        （条件は上の文章を参照）
        """
        wrong_text_tokens = tokenizer.tokenize(row['wrong_text'])
        correct_text_tokens = tokenizer.tokenize(row['correct_text'])
        if len(wrong_text_tokens) != len(correct_text_tokens):
            return False
        
        diff_count = 0
        threthold_count = 2
        for wrong_text_token, correct_text_token \
            in zip(wrong_text_tokens, correct_text_tokens):

            if wrong_text_token != correct_text_token:
                diff_count += 1
                if diff_count > threthold_count:
                    return False
        return True

    def normalize(text):
        """
        文字列の正規化
        """
        text = text.strip()
        text = unicodedata.normalize('NFKC', text)
        return text

# ここからデータセットの準備本体
    # 1 漢字の誤変換のデータのみを抜き出す。
    category_type = 'kanji-conversion'
    data_df.query('category == @category_type', inplace=True) 
    # 2 名前を変更
    data_df.rename(
        columns={'pre_text': 'wrong_text', 'post_text': 'correct_text'}, 
        inplace=True
    )
    
    # 3 誤変換と正しい文章をそれぞれ正規化し、
    # 4 それらの間でトークン列に対応がつくもののみを抜き出す。
    data_df['wrong_text'] = data_df['wrong_text'].map(normalize) 
    data_df['correct_text'] = data_df['correct_text'].map(normalize)
    kanji_conversion_num = len(data_df)
    data_df = data_df[data_df.apply(check_token_count, axis=1)]
    same_tokens_count_num = len(data_df)
    print(
        f'- 漢字誤変換の総数：{kanji_conversion_num}',
        f'- トークンの対応関係のつく文章の総数: {same_tokens_count_num}',
        f'  (全体の{same_tokens_count_num/kanji_conversion_num*100:.0f}%)',
        sep = '\n'
    )
    return data_df[['wrong_text', 'correct_text']].to_dict(orient='records')

# データのロード
train_df = pd.read_json(
    './jwtd/train.jsonl', orient='records', lines=True
)
test_df = pd.read_json(
    './jwtd/test.jsonl', orient='records', lines=True
)

# 学習用と検証用データ
print('学習と検証用のデータセット：')
dataset = create_dataset(train_df)
random.shuffle(dataset)
n = len(dataset)
n_train = int(n*0.8)
dataset_train = dataset[:n_train]
dataset_val = dataset[n_train:]

# テストデータ
print('テスト用のデータセット：')
dataset_test = create_dataset(test_df)

学習と検証用のデータセット：


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertJapaneseTokenizer'. 
The class this function is called from is 'SC_tokenizer'.


- 漢字誤変換の総数：235490
- トークンの対応関係のつく文章の総数: 171708
  (全体の73%)
テスト用のデータセット：


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertJapaneseTokenizer'. 
The class this function is called from is 'SC_tokenizer'.


- 漢字誤変換の総数：3061
- トークンの対応関係のつく文章の総数: 2252
  (全体の74%)


In [20]:
'''
ファインチューニング

まずは、データローダーを作成し
次にPyTorch Lightningでファインチューニングを行う

データローダ手順
    データセットを整形
    データセットの作成
    データローダーを定義

PyTorch Lightning
    学習データを読み込んでlossを返す関数の定義
    検証データを読み込んでlossを返す関数の定義
    configure_optimizers定義
    checkpoint(訓練する際の指標)作成
    訓練クラス(Trainer)作成
    ファインチューニング実行

'''

def create_dataset_for_loader(tokenizer, dataset, max_length):
    """
    データセットをデータローダに入力可能な形式にする。
    """
    dataset_for_loader = []
    for sample in tqdm(dataset):
        wrong_text = sample['wrong_text']
        correct_text = sample['correct_text']
        encoding = tokenizer.encode_plus_tagged(
            wrong_text, correct_text, max_length=max_length
        )
        encoding = { k: torch.tensor(v) for k, v in encoding.items() }
        dataset_for_loader.append(encoding)
    return dataset_for_loader

tokenizer = SC_tokenizer.from_pretrained(MODEL_NAME)

# データセットの作成
max_length = 32
dataset_train_for_loader = create_dataset_for_loader(
    tokenizer, dataset_train, max_length
)
dataset_val_for_loader = create_dataset_for_loader(
    tokenizer, dataset_val, max_length
)

# データローダの作成
# batch_sizeは３２と少なめ：その方が学習が早い
dataloader_train = DataLoader(
    dataset_train_for_loader, batch_size=32, shuffle=True
)
dataloader_val = DataLoader(dataset_val_for_loader, batch_size=256)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertJapaneseTokenizer'. 
The class this function is called from is 'SC_tokenizer'.
100%|██████████| 34342/34342 [00:21<00:00, 1607.55it/s]


In [22]:
# 学習
class BertForMaskedLM_pl(pl.LightningModule):
        
    def __init__(self, model_name, lr):
        super().__init__()
        self.save_hyperparameters()
        self.bert_mlm = BertForMaskedLM.from_pretrained(model_name)
        
    def training_step(self, batch, batch_idx):
        output = self.bert_mlm(**batch)
        loss = output.loss
        self.log('train_loss', loss)
        return loss
        
    def validation_step(self, batch, batch_idx):
        output = self.bert_mlm(**batch)
        val_loss = output.loss
        self.log('val_loss', val_loss)
   
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.hparams.lr)

checkpoint = pl.callbacks.ModelCheckpoint(
    monitor='val_loss',
    mode='min',
    save_top_k=1,
    save_weights_only=True,
    dirpath='model/'
)

trainer = pl.Trainer(
    gpus=1,
    # max_epochs=5,
    # 5だと1~2時間かかるらしい
    max_epochs=1,
    callbacks=[checkpoint]
)

# ファインチューニング
model = BertForMaskedLM_pl(MODEL_NAME, lr=1e-5)
trainer.fit(model, dataloader_train, dataloader_val)
best_model_path = checkpoint.best_model_path


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True, used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
INFO:pytorch_lightni

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [23]:
# 推測
def predict(text, tokenizer, bert_mlm):
    """
    文章を入力として受け、BERTが予測した文章を出力
    """
    # 符号化
    encoding, spans = tokenizer.encode_plus_untagged(
        text, return_tensors='pt'
    ) 
    encoding = { k: v.cuda() for k, v in encoding.items() }

    # ラベルの予測値の計算
    with torch.no_grad():
        output = bert_mlm(**encoding)
        scores = output.logits
        labels_predicted = scores[0].argmax(-1).cpu().numpy().tolist()

    # ラベル列を文章に変換
    predict_text = tokenizer.convert_bert_output_to_text(
        text, labels_predicted, spans
    )

    return predict_text

# いくつかの例に対してBERTによる文章校正を行ってみる。
text_list = [
    'ユーザーの試行に合わせた楽曲を配信する。',
    'メールに明日の会議の史料を添付した。',
    '乳酸菌で牛乳を発行するとヨーグルトができる。',
    '突然、子供が帰省を発した。'
]

# トークナイザ、ファインチューニング済みのモデルのロード
tokenizer = SC_tokenizer.from_pretrained(MODEL_NAME)
model = BertForMaskedLM_pl.load_from_checkpoint(best_model_path)
bert_mlm = model.bert_mlm.cuda()

for text in text_list:
    predict_text = predict(text, tokenizer, bert_mlm) # BERTによる予測
    print('---')
    print(f'入力：{text}')
    print(f'出力：{predict_text}')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertJapaneseTokenizer'. 
The class this function is called from is 'SC_tokenizer'.
Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


---
入力：ユーザーの試行に合わせた楽曲を配信する。
出力：ユーザーの嗜好に合わせた楽曲を配信する。
---
入力：メールに明日の会議の史料を添付した。
出力：メールに明日の会議の史料を添付した。
---
入力：乳酸菌で牛乳を発行するとヨーグルトができる。
出力：乳酸菌で牛乳を発酵するとヨーグルトができる。
---
入力：突然、子供が帰省を発した。
出力：突然、子供が帰省を発した。


In [24]:
# BERTで予測を行い、正解数を数える。
correct_num = 0 
for sample in tqdm(dataset_test):
    wrong_text = sample['wrong_text']
    correct_text = sample['correct_text']
    predict_text = predict(wrong_text, tokenizer, bert_mlm) # BERT予測
   
    if correct_text == predict_text: # 正解の場合
        correct_num += 1

print(f'Accuracy: {correct_num/len(dataset_test):.2f}')

100%|██████████| 2252/2252 [00:28<00:00, 78.82it/s]

Accuracy: 0.69


In [25]:
# 正しく誤変換の位置を特定できた割合を評価
# 正しい漢字にたとえ直せなかったとしても、位置を特定するだけで価値はある（wordの赤線)

correct_position_num = 0 # 正しく誤変換の漢字を特定できたデータの数
for sample in tqdm(dataset_test):
    wrong_text = sample['wrong_text']
    correct_text = sample['correct_text']
    
    # 符号化
    encoding = tokenizer(wrong_text)
    wrong_input_ids = encoding['input_ids'] # 誤変換の文の符合列
    encoding = {k: torch.tensor([v]).cuda() for k,v in encoding.items()}
    correct_encoding = tokenizer(correct_text)
    correct_input_ids = correct_encoding['input_ids'] # 正しい文の符合列
    
    # 文章を予測
    with torch.no_grad():
        output = bert_mlm(**encoding)
        scores = output.logits
        # 予測された文章のトークンのID
        predict_input_ids = scores[0].argmax(-1).cpu().numpy().tolist() 

    # 特殊トークンを取り除く
    wrong_input_ids = wrong_input_ids[1:-1]
    correct_input_ids =  correct_input_ids[1:-1]
    predict_input_ids =  predict_input_ids[1:-1]
    
    # 誤変換した漢字を特定できているかを判定
    # 符合列を比較する。
    detect_flag = True
    for wrong_token, correct_token, predict_token \
        in zip(wrong_input_ids, correct_input_ids, predict_input_ids):

        if wrong_token == correct_token: # 正しいトークン
            # 正しいトークンなのに誤って別のトークンに変換している場合
            if wrong_token != predict_token: 
                detect_flag = False
                break
        else: # 誤変換のトークン
            # 誤変換のトークンなのに、そのままにしている場合
            if wrong_token == predict_token: 
                detect_flag = False
                break

    if detect_flag: # 誤変換の漢字の位置を正しく特定できた場合
        correct_position_num += 1
        
print(f'Accuracy: {correct_position_num/len(dataset_test):.2f}')

100%|██████████| 2252/2252 [00:29<00:00, 77.61it/s]

Accuracy: 0.76
